In [2]:
import torch
import torch.nn as nn

In [4]:
# Input (temp, rainfall, humidity)
inputs = torch.tensor([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype=torch.float32)

# Targets (apples, oranges)
targets = torch.tensor([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype=torch.float32)

In [5]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [6]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

TensorDataset allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [7]:
from torch.utils.data import TensorDataset

In [8]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

A DataLoader splits the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [9]:
from torch.utils.data import DataLoader

In [10]:
# Define data loader
batch_size = 5
train_data = DataLoader(train_ds, batch_size, shuffle=True)

In [11]:
for X, y in train_data:
    print(X)
    print(y)
    break

tensor([[101.,  44.,  37.],
        [ 92.,  87.,  64.],
        [ 68.,  96.,  71.],
        [ 91.,  87.,  65.],
        [103.,  43.,  36.]])
tensor([[ 21.,  38.],
        [ 82., 100.],
        [104., 118.],
        [ 80., 102.],
        [ 20.,  38.]])


### nn.Linear()

    #Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically

In [15]:
# Define model
linear_model = nn.Linear(3, 2)
print(linear_model.weight)
print(linear_model.bias)

Parameter containing:
tensor([[-0.2389,  0.2124,  0.2872],
        [ 0.2187,  0.5588, -0.0950]], requires_grad=True)
Parameter containing:
tensor([0.0496, 0.1226], requires_grad=True)


In [17]:
# Parameters
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.2389,  0.2124,  0.2872],
         [ 0.2187,  0.5588, -0.0950]], requires_grad=True),
 Parameter containing:
 tensor([0.0496, 0.1226], requires_grad=True)]

In [18]:
# Generate predictions
y_pred = linear_model(inputs)
y_pred

tensor([[ 9.1876, 49.4421],
        [15.3777, 63.1187],
        [24.3790, 88.5198],
        [-4.5596, 42.9415],
        [24.0550, 62.2087],
        [ 8.7363, 49.1019],
        [15.4525, 62.4649],
        [24.4273, 88.6435],
        [-4.1084, 43.2817],
        [24.5810, 61.8951],
        [ 9.2624, 48.7883],
        [14.9265, 62.7785],
        [24.3042, 89.1736],
        [-5.0857, 43.2552],
        [24.5062, 62.5489]], grad_fn=<AddmmBackward>)

In [19]:
# Import nn.functional
import torch.nn.functional as F

In [20]:
# Define loss function
loss_fn = F.mse_loss

In [22]:
loss = loss_fn(linear_model(inputs), targets)
print(loss)

tensor(2933.7275, grad_fn=<MseLossBackward>)


In [23]:
# Define optimizer
opt = torch.optim.SGD(linear_model.parameters(), lr=1e-5)

In [24]:
# Utility function to train the model

def train_utility(num_epochs, model, loss_fn, opt, train_data):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for X,y in train_data:
            
            # 1. Generate predictions
            y_pred = model(X)
            
            # 2. Calculate loss
            loss = loss_fn(y_pred, y)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [26]:
train_utility(300, linear_model, loss_fn, opt, train_data)

Epoch [10/300], Loss: 282.0091
Epoch [20/300], Loss: 140.4294
Epoch [30/300], Loss: 55.5235
Epoch [40/300], Loss: 102.6329
Epoch [50/300], Loss: 60.4891
Epoch [60/300], Loss: 64.7169
Epoch [70/300], Loss: 32.5302
Epoch [80/300], Loss: 59.3686
Epoch [90/300], Loss: 34.0054
Epoch [100/300], Loss: 41.7797
Epoch [110/300], Loss: 26.0860
Epoch [120/300], Loss: 32.2542
Epoch [130/300], Loss: 28.2498
Epoch [140/300], Loss: 22.7104
Epoch [150/300], Loss: 27.8591
Epoch [160/300], Loss: 9.2141
Epoch [170/300], Loss: 6.5895
Epoch [180/300], Loss: 18.0521
Epoch [190/300], Loss: 8.7308
Epoch [200/300], Loss: 16.7496
Epoch [210/300], Loss: 8.0196
Epoch [220/300], Loss: 15.9278
Epoch [230/300], Loss: 13.9257
Epoch [240/300], Loss: 14.9016
Epoch [250/300], Loss: 8.0914
Epoch [260/300], Loss: 16.0742
Epoch [270/300], Loss: 11.0364
Epoch [280/300], Loss: 11.1286
Epoch [290/300], Loss: 5.1676
Epoch [300/300], Loss: 8.4411


In [27]:
# Generate predictions
y_pred = linear_model(inputs)
y_pred

tensor([[ 57.0610,  70.8804],
        [ 81.5766,  98.2030],
        [119.2499, 138.3338],
        [ 21.1999,  39.3875],
        [100.8403, 113.4363],
        [ 55.8056,  69.7487],
        [ 81.3409,  97.8273],
        [119.4942, 138.6792],
        [ 22.4554,  40.5192],
        [101.8601, 114.1923],
        [ 56.8253,  70.5048],
        [ 80.3212,  97.0713],
        [119.4856, 138.7095],
        [ 20.1802,  38.6314],
        [102.0957, 114.5680]], grad_fn=<AddmmBackward>)

In [28]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])